In [95]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
%matplotlib inline

In [96]:
data = pd.read_csv('/Users/logno/Documents/GitHub/diabetes-project/diabetes.csv')
df = data.copy()

# Broad overview of the data

In [97]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [98]:
df.shape

(768, 9)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [100]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [101]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Cleaning

Despite there being no N/A values in this dataset, I believe that there are some values in certain columns that are incorrect. For example, it is impossible for a person to have an insulin value of zero, and if they did, that would be an issue. In this section, I will clean up these values and ensure that all of the data makes sense.

## Finding which columns to check

In [102]:
df.loc[df['Glucose'] == 0].shape

(5, 9)

In [103]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [104]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [105]:
df.loc[df['Age'] == 0].shape

(0, 9)

In [106]:
df.loc[df['DiabetesPedigreeFunction'] == 0].shape

(0, 9)

In [107]:
bad_insulin = df.loc[df['Insulin'] == 0]
ins = bad_insulin[['Insulin', 'Outcome']]
ins['Outcome'].sum()

138

It seems that the columns with the missing values are glucose, bloodpressure, and BMI. 

## Glucose

In [108]:
# Finding the rows that have a glucose of 0
df.loc[df['Glucose'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


Since there are only 5 rows, I think that it would be best to drop them.

In [109]:
# Dropping the problem rows and creating a new dataframe
df = df.drop(df.index[[75,182,342,349,502]])

In [110]:
# Checking if the values were dropped
df.loc[df['Glucose'] == 0].shape

(0, 9)

## Insulin

In [111]:
df.loc[df['Insulin'] == 0].shape

(370, 9)

In [112]:
# Finding how many null values there are
df['Insulin'] = df['Insulin'].replace({0: np.nan})
df['Insulin'].isnull().sum()

370

In [113]:
# Imputing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')

df["Insulin"] = imp.fit_transform(df[["Insulin"]]).ravel()
df['Insulin'].isnull().sum()

0

## Blood Pressure

In [114]:
df['BloodPressure'].describe()

count    763.000000
mean      69.115334
std       19.399240
min        0.000000
25%       62.000000
50%       72.000000
75%       80.000000
max      122.000000
Name: BloodPressure, dtype: float64

In [115]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [116]:
# Finding how many null values there are
df['BloodPressure'] = df['BloodPressure'].replace({0: np.nan})
df['BloodPressure'].isnull().sum()

35

Since there are a fair number of null values, I will impute them using the median.

In [117]:
# Imputing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')

df["BloodPressure"] = imp.fit_transform(df[["BloodPressure"]]).ravel()
df['BloodPressure'].isnull().sum()

0

## BMI

In [118]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [119]:
df['BMI'].describe()

count    763.000000
mean      31.986763
std        7.893221
min        0.000000
25%       27.300000
50%       32.000000
75%       36.550000
max       67.100000
Name: BMI, dtype: float64

I will impute these values as well, just for fun. I can come back to this later.

In [120]:
# Imputing values
df["BMI"] = imp.fit_transform(df[["BMI"]]).ravel()
df['BMI'].isnull().sum()

0

Time to double check that I am done cleaning.

In [121]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [122]:
df.shape

(763, 9)

# Building models

In [123]:
# Creating my splits
from sklearn.model_selection import train_test_split

y = df.Outcome.values
X = df.drop(['Outcome'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=27)

In [124]:
# Scaling data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X =  pd.DataFrame(sc_X.fit_transform(df.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

## RandomForestClassifier

In [125]:
# Building the base model
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(random_state=27)
rfm.fit(X_train, y_train.ravel())

RandomForestClassifier(random_state=27)

In [126]:
# Finding the model's predictions
predictions = rfm.predict(X_test)

from sklearn import metrics

print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, predictions)))

Accuracy = 0.771


I got an accuracy of 78%, which is not bad. I will test some other models and compare.

## XGBoostClassifier

In [127]:
# Building the model
import xgboost as xgb
xgb_classif = xgb.XGBClassifier()
xgb_classif = xgb_classif.fit(X_train, y_train)

print('The accuracy of the xgboost classifier is {:.2f} out of 1 on the training data'.format(xgb_classif.score(X_train, y_train)))
print('The accuracy of the xgboost classifier is {:.2f} out of 1 on the test data'.format(xgb_classif.score(X_test, y_test)))

The accuracy of the xgboost classifier is 1.00 out of 1 on the training data
The accuracy of the xgboost classifier is 0.78 out of 1 on the test data


XGBoost gave me an accuracy of 75%, not too bad either.

## KNN Classifier

In [128]:
# Building the model
from sklearn.neighbors import KNeighborsClassifier

test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [129]:
## score that comes from testing on the same datapoints that were used for training
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

Max train score 100.0 % and k = [1]


In [130]:
## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

Max test score 76.47058823529412 % and k = [9]


In [131]:
#Setup a knn classifier with k neighbors
knn = KNeighborsClassifier(14)

knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.7516339869281046

In [132]:
#import confusion_matrix
from sklearn.metrics import confusion_matrix
#let us get the predictions using the classifier we had fit above
y_pred = knn.predict(X_test)
confusion_matrix(y_test,y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,86,10,96
1,28,29,57
All,114,39,153


In [133]:
# Classification_report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

precision    recall  f1-score   support

           0       0.75      0.90      0.82        96
           1       0.74      0.51      0.60        57

    accuracy                           0.75       153
   macro avg       0.75      0.70      0.71       153
weighted avg       0.75      0.75      0.74       153



The KNN Classifier got an accuracy of 76% with mostly true positives, which is good, but the random forest classifier still scored higher. I will now hyperparameter this model.

## Hyperparameters: RandomForestClassifier

In [134]:
# Running randomsearch to find best parameters
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50,100, 300, 500, 800, 1200]
max_depth = [1, 2 ,3 ,5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 50, 100]
min_samples_leaf = [1, 2, 5, 10, 20] 

hyperF = dict(n_estimators = n_estimators, 
              max_depth = max_depth,  
              min_samples_split = min_samples_split, 
              min_samples_leaf = min_samples_leaf)

rs = RandomizedSearchCV(rfm, hyperF, cv=4, scoring='accuracy',verbose=10)
rs.fit(X_train, y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25, score=0.758, total=   1.7s
[CV] n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[CV]  n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25, score=0.765, total=   1.8s
[CV] n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.5s remaining:    0.0s
[CV]  n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25, score=0.796, total=   1.9s
[CV] n_estimators=1200, min_samples_split=50, min_samples_leaf=10, max_depth=25 
[Parallel(n_jobs=1)]: Done   3 out of   3 | ela

RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(random_state=27),
                   param_distributions={'max_depth': [1, 2, 3, 5, 8, 15, 25,
                                                      30],
                                        'min_samples_leaf': [1, 2, 5, 10, 20],
                                        'min_samples_split': [2, 5, 10, 15, 50,
                                                              100],
                                        'n_estimators': [50, 100, 300, 500, 800,
                                                         1200]},
                   scoring='accuracy', verbose=10)

In [135]:
# Finding the accuracy of the refined model
rs.best_score_

0.7737680598555211

In [139]:
# Building the improved model
rfm = RandomForestClassifier(max_depth=25, min_samples_leaf=2, n_estimators=800,
                       random_state=10)
rfm.fit(X_train, y_train.ravel())

preds = rfm.predict(X_test)

from sklearn.model_selection import cross_val_score

score = cross_val_score(rfm, X_test, y_test.ravel(), cv = 10)
score.mean()

0.7720833333333333

## Gradient Boost Classifier

In [141]:
# Building the model
from sklearn.ensemble import GradientBoostingClassifier

gbe = GradientBoostingClassifier(random_state=27)
parameters={
    'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.5, 1],
    'max_depth': [3, 4, 5]
}
gridsearch=GridSearchCV(gbe,parameters,cv=100,scoring='roc_auc')
gridsearch.fit(X_train,y_train)
print(gridsearch.best_params_)
print(gridsearch.best_score_)

{'learning_rate': 0.05, 'max_depth': 4, 'max_features': 1}
0.8318333333333334


In [189]:
# The improved model
final_model = GradientBoostingClassifier(learning_rate=0.05,max_depth=4,max_features=1,random_state=27)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20,random_state=27)
final_model.fit(x_train,y_train)

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

yprediction = final_model.predict_proba(x_test)[:,1]
score = round(roc_auc_score(y_test,yprediction),5)
score

0.85216

The Gradient Boost Classifier did much better than any of the other models! It is safe to say that I will be using this model for production.

# Saving the model

In [190]:
import pickle
saved_model = pickle.dumps(final_model) 

In [191]:
model_from_pickle = pickle.loads(saved_model) 

In [192]:
model_from_pickle.predict(x_test) 

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0])

In [193]:
y_test

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0])